In [103]:
# load packages
import numpy as np
import scipy as sp
import pandas as pd
from scipy import optimize as opt
from scipy.integrate import odeint 
from scipy.optimize import minimize

'c:\\Users\\chloe\\Desktop\\ChloeYou\\UBC_grad\\Term3\\MATH561\\math561-gp-SuEIR\\code'

Now consider parameter learning for the SuEIR model. Given the model parameters $\boldsymbol{\theta}$ and initial quantities $S_0, E_0, I_0$, and $R_0$, we can compute the number of individuals in each group (i.e., $S, E, I$, and $R$ ) at time $t$, denoted by $\widehat{S}_t, \widehat{E}_t, \widehat{I}_t$ and $\widehat{R}_t$, via applying  numerical ODE solvers onto the ODE. Then we propose to learn the model parameter $\widehat{\boldsymbol{\theta}}=(\widehat{\beta}, \widehat{\sigma}, \widehat{\gamma}, \widehat{\mu})$ by minimizing the following logarithmic-type mean square error (MSE).

Where $\mathbf{I}=\left\{I_t\right\}_{t=1}^T, \mathbf{R}=\left\{R_t\right\}_{t=1}^T$ with $I_t$ and $R_t$ denote the reported numbers of infected cases and removed cases (including both recovered cases and fatality cases) at time $t$ (i.e., date), and $p$ is the smoothing parameter used to ensure numerical stability. Note that given $S_0, E_0, I_0$ and $R_0, \widehat{I}_t$ and $\widehat{R}_t$ can be described as differentiable functions of the parameter $\boldsymbol{\theta}$. Then the model parameter $\widehat{\boldsymbol{\theta}}=\operatorname{argmin}_{\boldsymbol{\theta}} L(\boldsymbol{\theta} ; \mathbf{I}, \mathbf{R})$ can be learnt by applying standard gradient based optimizer (e.g., BFGS) onto the loss function under the constraint that $\beta, \sigma, \gamma, \mu \in[0,1]$.

$$
L(\boldsymbol{\theta} ; \mathbf{I}, \mathbf{R})=\frac{1}{T} \sum_{t=1}^T\left[\left(\log \left(\widehat{I}_t+p\right)-\log \left(I_t+p\right)\right)^2+\left(\log \left(\widehat{R}_t+p\right)-\log \left(R_t+p\right)\right)^2\right],
$$


In [145]:
def loss(pred, target, smoothing=20): 
    return np.mean((np.log(pred+smoothing) - np.log(target+smoothing))**2)
    
# SEIR: the 'reported' SEIR. In our case, the simulated dataset. 
def optim(SEIR):
    # maybe these could be passed into the function as  
    # parameters if the estimation parts change
    y0 = [800.0, 100.0, 50.0, 50.0]
    t = np.linspace(0, 59, 60)
    N  = 1000.0  
    def deriv(y, t, N, beta, mu, sigma, gamma):
        s,e,i,r= y
        dsdt = -((beta * (e + i) * s) / N)
        dedt = (((beta * (e + i) * s) / N) - sigma * e)
        didt = (mu * sigma * e - (gamma * i))
        drdt = (gamma * i)  
        return dsdt, dedt, didt, drdt
    
    def objective(x):
        beta, mu, sigma, gamma = x
        est_SEIR = odeint(deriv, y0, t, args=(N, beta, mu, sigma, gamma))
        est_df = pd.DataFrame(est_SEIR, columns = ['S','E','I','R'])
        return loss(est_df.loc[:,"I"], SEIR.loc[:,"I"]) +  loss(est_df.loc[:,"R"], SEIR.loc[:,"R"])

    # scipy optimizer
    optimal = minimize(
        objective,
        [0.2, .5e-2, 2.5e-1, 0.01], # initial estimate
        method='Nelder-Mead',
        bounds=[(0.0001, .3), (0.001, 0.3), (0.01, 1), (0.001, 1.)]
    )

    return optimal.x

Load datasets and then minimize the objective function and obtain parameter estimates:

In [146]:
os.chdir('c:\\Users\\chloe\\Desktop\\ChloeYou\\UBC_grad\\Term3\\MATH561\\math561-gp-SuEIR\\data\\python')

params_trad = pd.DataFrame([])
for file in os.listdir(os.getcwd()) :
    df = pd.read_csv(file)
    params_trad[file] = optim(df)
    
params_trad # rows: beta, mu, sigma, gamma


,sim-1-p.csv,sim-10-p.csv,sim-2-p.csv,sim-3-p.csv,sim-4-p.csv,sim-5-p.csv,sim-6-p.csv,sim-7-p.csv,sim-8-p.csv,sim-9-p.csv
0,0.300000,0.300000,0.199354,0.300000,0.300000,0.300000,0.300000,0.300000,0.137262,0.181348
1,0.001000,0.299965,0.299999,0.299999,0.001000,0.213042,0.001000,0.001000,0.299482,0.299996
2,0.159738,0.033030,0.055424,0.028412,0.103648,1.000000,0.119547,0.121854,0.062290,0.051656
3,0.079006,0.095723,0.083540,0.110760,0.049379,0.072733,0.060722,0.062337,0.116919,0.095466


In [147]:
os.chdir('c:\\Users\\chloe\\Desktop\\ChloeYou\\UBC_grad\\Term3\\MATH561\\math561-gp-SuEIR\\data\\matlab')

csv = os.listdir(os.getcwd())
del csv[0] # this is the ground truth so we will only calculate the R_0 from it
print(csv)
params_nn = pd.DataFrame([])
for file in csv :
    df = pd.read_csv(file)
    params_nn[file] = optim(df)
    
params_nn # rows: beta, mu, sigma, gamma

['sim-1-m.csv', 'sim-10-m.csv', 'sim-2-m.csv', 'sim-3-m.csv', 'sim-4-m.csv', 'sim-5-m.csv', 'sim-6-m.csv', 'sim-7-m.csv', 'sim-8-m.csv', 'sim-9-m.csv']


,sim-1-m.csv,sim-10-m.csv,sim-2-m.csv,sim-3-m.csv,sim-4-m.csv,sim-5-m.csv,sim-6-m.csv,sim-7-m.csv,sim-8-m.csv,sim-9-m.csv
0,0.300000,0.300000,0.300000,0.300000,0.300000,0.300000,0.300000,0.300000,0.300000,0.300000
1,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000,0.001000
2,0.104246,0.080340,0.100045,0.092735,0.080929,0.077680,0.111918,0.108472,0.102347,0.090491
3,0.070776,0.076049,0.070245,0.070619,0.075701,0.095788,0.054076,0.052862,0.051843,0.053977


Calculate R_0 for each simulation:
$$
R_0 = \frac{\beta}{\sigma} +\frac{\beta\mu}{\gamma}
$$

In [143]:
R_0_trad = (params_trad.loc[0,]/params_trad.loc[2,]) + (params_trad.loc[0,]*params_trad.loc[1,]/params_trad.loc[3,])


sim-1-p.csv      1.881874
sim-10-p.csv    10.022770
sim-2-p.csv      4.312780
sim-3-p.csv     11.371481
sim-4-p.csv      2.900479
sim-5-p.csv      1.178727
sim-6-p.csv      2.514407
sim-7-p.csv      2.466778
sim-8-p.csv      2.555172
sim-9-p.csv      4.080544
dtype: float64

In [144]:
R_0_nn = (params_nn.loc[0,]/params_nn.loc[2,]) + (params_nn.loc[0,]*params_nn.loc[1,]/params_nn.loc[3,])

sim-1-m.csv     2.882059
sim-10-m.csv    3.738078
sim-2-m.csv     3.002921
sim-3-m.csv     3.239290
sim-4-m.csv     3.710916
sim-5-m.csv     3.865127
sim-6-m.csv     2.686081
sim-7-m.csv     2.771377
sim-8-m.csv     2.936978
sim-9-m.csv     3.320800
dtype: float64

 Let's compare these estimated values to the 'true' $R_0 = \frac{0.1}{0.09} +\frac{0.1*0.1}{0.12} = 1.19$ 

The traditional method gives $R_0$ estimates that vary more, although in some simulation it was able to recover an $R_0$ very close to the 'true' $R_0$. Although it seems like the simulated 'reported' cases that was estimated from the artificial neural network ODE numerical solver gives more reasonable $R_0$ estimates - ranging around 2 and 3.